# Управление кредитным портфелем на основе модели Мертона

Этот ноутбук демонстрирует систему управления кредитным портфелем с использованием:
- **Модель Мертона** для расчета вероятности дефолта (PD)
- **Макроэкономические факторы** для корректировки риска
- **Автоматизированная система** принятия кредитных решений
- **Оптимизация портфеля** на основе риск-доходности

## Цели проекта:
1. Рассчитать PD для каждого потенциального заемщика
2. Установить лимиты кредитного риска
3. Автоматизировать процесс оценки кредитных заявок
4. Оптимизировать состав кредитного портфеля
5. Мониторинг и управление рисками портфеля

## 1. Импорт необходимых библиотек

In [ ]:
# Основные библиотеки для работы с данными и моделирования
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Библиотеки для машинного обучения и статистики
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from scipy import stats
from scipy.optimize import minimize

# Импорт нашего класса для управления портфелем
import sys
sys.path.append('./utils')
from portfolio import Portfolio
from load_data import load_stocks_data, load_macro_data
from logger import log

# Настройка стиля графиков
plt.style.use('default')
sns.set_palette("husl")

print("✅ Все библиотеки успешно импортированы")

## 2. Загрузка и подготовка данных для кредитного портфеля

In [ ]:
# Список компаний для кредитного портфеля (потенциальные заемщики)
credit_tickers = [
    'GAZP',   # Газпром (Нефтегаз)
    'SBER',   # Сбербанк (Финансы) 
    'LKOH',   # Лукойл (Нефтегаз)
    'GMKN',   # Норникель (Металлургия)
    'VTBR',   # ВТБ (Финансы)
    'ROSN',   # Роснефть (Нефтегаз)
    'NLMK',   # НЛМК (Металлургия)
    'MTSS',   # МТС (Телеком)
    'MGNT',   # Магнит (Ритейл)
    'MOEX',   # Московская биржа (Финансы)
    'RTKM',   # Ростелеком (Телеком)
    'RUAL',   # РУСАЛ (Металлургия)
    'LNTA',   # Лента (Ритейл)
    'FESH',   # Дальневосточная энергетическая компания (Ритейл)
    'VTBR'    # ВТБ (Финансы)
]

# Создание экземпляра портфеля
print("📊 Создание кредитного портфеля...")
credit_portfolio = Portfolio(credit_tickers)

# Загрузка данных компаний
print("📈 Загрузка данных компаний...")
credit_portfolio.load_stocks_data()

# Загрузка макроэкономических данных
print("🌍 Загрузка макроэкономических данных...")
credit_portfolio.load_macro_data()

print(f"✅ Данные загружены для {len(credit_tickers)} компаний")
print(f"📅 Период данных: {credit_portfolio.portfolio['date'].min()} - {credit_portfolio.portfolio['date'].max()}")
print(f"📊 Общее количество наблюдений: {len(credit_portfolio.portfolio)}")

# Просмотр структуры данных
credit_portfolio.portfolio.head()

## 3. Расчет вероятности дефолта (PD) по модели Мертона

In [ ]:
# Расчет PD по модели Мертона с учетом макроэкономических факторов
print("⚡ Расчет вероятности дефолта по модели Мертона...")

# Добавляем расчет PD в портфель
credit_portfolio.add_merton_pd()

# Анализируем полученные PD
pd_analysis = credit_portfolio.portfolio.groupby('ticker')['PD'].agg([
    'mean', 'std', 'min', 'max', 'count'
]).round(4)

print("📊 Статистика PD по компаниям:")
print(pd_analysis)

# Визуализация распределения PD
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Гистограмма всех PD
axes[0,0].hist(credit_portfolio.portfolio['PD'], bins=50, alpha=0.7, color='skyblue')
axes[0,0].axvline(credit_portfolio.portfolio['PD'].mean(), color='red', linestyle='--', label=f'Среднее: {credit_portfolio.portfolio["PD"].mean():.3f}')
axes[0,0].set_xlabel('Probability of Default (PD)')
axes[0,0].set_ylabel('Частота')
axes[0,0].set_title('Распределение вероятности дефолта')
axes[0,0].legend()

# 2. Boxplot PD по компаниям
pd_by_company = [credit_portfolio.portfolio[credit_portfolio.portfolio['ticker']==ticker]['PD'].values 
                for ticker in credit_tickers[:8]]  # Первые 8 для наглядности
axes[0,1].boxplot(pd_by_company, labels=credit_tickers[:8])
axes[0,1].set_xticklabels(credit_tickers[:8], rotation=45)
axes[0,1].set_ylabel('PD')
axes[0,1].set_title('Распределение PD по компаниям')

# 3. Временной ряд средней PD
monthly_pd = credit_portfolio.portfolio.groupby('date')['PD'].mean()
axes[1,0].plot(monthly_pd.index, monthly_pd.values, marker='o', linewidth=2)
axes[1,0].set_xlabel('Дата')
axes[1,0].set_ylabel('Средняя PD')
axes[1,0].set_title('Динамика средней PD портфеля')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Корреляция PD с макропоказателями
macro_corr = credit_portfolio.portfolio[['PD', 'inflation', 'unemployment', 'usd_rub']].corr()['PD'].drop('PD')
axes[1,1].bar(range(len(macro_corr)), macro_corr.values)
axes[1,1].set_xticks(range(len(macro_corr)))
axes[1,1].set_xticklabels(['Инфляция', 'Безработица', 'USD/RUB'], rotation=45)
axes[1,1].set_ylabel('Корреляция с PD')
axes[1,1].set_title('Корреляция PD с макрофакторами')

plt.tight_layout()
plt.show()

print(f"✅ PD рассчитана. Средняя PD портфеля: {credit_portfolio.portfolio['PD'].mean():.3f} ({credit_portfolio.portfolio['PD'].mean()*100:.2f}%)")

## 4. Установка лимитов кредитного риска

In [ ]:
# Устанавливаем лимиты кредитного риска
print("🎯 Установка лимитов кредитного риска...")

# Создаем консервативные лимиты для кредитного банка
credit_portfolio.create_credit_risk_limits(
    max_pd_threshold=0.05,      # Максимальная PD для одобрения кредита: 5%
    max_sector_concentration=0.25,  # Максимальная концентрация в секторе: 25%
    max_single_exposure=0.08    # Максимальная доля одного заемщика: 8%
)

print("✅ Лимиты установлены:")
for limit_name, limit_value in credit_portfolio.credit_limits.items():
    if limit_name != 'total_portfolio_limit':
        print(f"   • {limit_name}: {limit_value:.1%}")

# Анализ текущего состояния портфеля относительно лимитов
current_pd_stats = credit_portfolio.portfolio.groupby('ticker')['PD'].last()

print(f"\n📊 Анализ соответствия лимитам:")
print(f"Компании с PD ≤ 5%: {(current_pd_stats <= 0.05).sum()}/{len(current_pd_stats)} ({(current_pd_stats <= 0.05).mean():.1%})")
print(f"Компании с PD > 5%: {(current_pd_stats > 0.05).sum()}/{len(current_pd_stats)} ({(current_pd_stats > 0.05).mean():.1%})")

# Показываем компании с высоким риском
high_risk_companies = current_pd_stats[current_pd_stats > 0.05]
if len(high_risk_companies) > 0:
    print(f"\n⚠️  Компании с PD > 5% (высокий риск):")
    for ticker, pd_value in high_risk_companies.items():
        print(f"   • {ticker}: PD = {pd_value:.3f} ({pd_value*100:.2f}%)")
else:
    print(f"\n✅ Все компании соответствуют лимиту PD ≤ 5%")

## 5. Система принятия кредитных решений

In [ ]:
# Демонстрация процесса принятия кредитных решений
print("🏦 Демонстрация системы принятия кредитных решений\n")

# Создаем список заявок на кредиты
sample_applications = [
    {'ticker': 'SBER', 'amount': 50000000, 'requested_rate': 0.09},   # 50 млн руб, 9%
    {'ticker': 'GAZP', 'amount': 100000000, 'requested_rate': 0.10},  # 100 млн руб, 10%
    {'ticker': 'VTBR', 'amount': 80000000, 'requested_rate': 0.12},   # 80 млн руб, 12%
    {'ticker': 'GMKN', 'amount': 150000000, 'requested_rate': 0.11},  # 150 млн руб, 11%
    {'ticker': 'LNTA', 'amount': 30000000, 'requested_rate': 0.15},   # 30 млн руб, 15%
]

# Размер текущего кредитного портфеля
portfolio_size = 1000000000  # 1 млрд рублей

# Оцениваем каждую заявку
loan_decisions = []
print("📋 АНАЛИЗ КРЕДИТНЫХ ЗАЯВОК:")
print("=" * 80)

for i, app in enumerate(sample_applications, 1):
    print(f"\n{i}. Заявка от {app['ticker']}:")
    print(f"   Сумма кредита: {app['amount']:,} руб. ({app['amount']/1e6:.1f} млн)")
    print(f"   Запрашиваемая ставка: {app['requested_rate']:.1%}")
    
    # Оцениваем заявку
    decision = credit_portfolio.assess_credit_application(
        borrower_ticker=app['ticker'],
        loan_amount=app['amount'],
        current_portfolio_size=portfolio_size
    )
    
    loan_decisions.append(decision)
    
    # Выводим результат
    status_emoji = "✅" if decision['decision'] == 'ОДОБРИТЬ' else "⚠️" if decision['decision'] == 'УСЛОВНО ОДОБРИТЬ' else "❌"
    print(f"   {status_emoji} РЕШЕНИЕ: {decision['decision']}")
    print(f"   📊 PD заемщика: {decision['pd']:.3f} ({decision['pd']*100:.2f}%)")
    print(f"   🏆 Рейтинг риска: {decision['risk_rating']}")
    print(f"   💰 Рекомендуемая ставка: {decision['recommended_rate']:.1%}")
    print(f"   📈 Доля в портфеле: {decision['exposure_ratio']:.2%}")
    print(f"   🏭 Сектор: {decision['sector']}")
    print(f"   💡 Обоснование: {decision['reason']}")

# Сводная статистика по решениям
approved = [d for d in loan_decisions if d['decision'] == 'ОДОБРИТЬ']
rejected = [d for d in loan_decisions if d['decision'] == 'ОТКЛОНИТЬ']  
conditional = [d for d in loan_decisions if d['decision'] == 'УСЛОВНО ОДОБРИТЬ']

print(f"\n" + "=" * 80)
print("📊 СВОДКА ПО КРЕДИТНЫМ РЕШЕНИЯМ:")
print(f"✅ Одобрено: {len(approved)} заявок")
print(f"❌ Отклонено: {len(rejected)} заявок")
print(f"⚠️  Условно одобрено: {len(conditional)} заявок")

if approved:
    total_approved = sum(d['loan_amount'] for d in approved)
    avg_pd = sum(d['pd'] for d in approved) / len(approved)
    avg_rate = sum(d['recommended_rate'] for d in approved) / len(approved)
    print(f"\n💰 Общая сумма одобренных кредитов: {total_approved:,} руб. ({total_approved/1e6:.1f} млн)")
    print(f"📊 Средняя PD одобренных: {avg_pd:.3f} ({avg_pd*100:.2f}%)")
    print(f"💹 Средняя рекомендуемая ставка: {avg_rate:.2%}")
    print(f"📈 Использование лимита портфеля: {total_approved/portfolio_size:.1%}")

## 6. Оптимизация кредитного портфеля

In [ ]:
# Оптимизация состава кредитного портфеля
print("🎯 ОПТИМИЗАЦИЯ КРЕДИТНОГО ПОРТФЕЛЯ\n")

# Создаем расширенный список заявок для оптимизации
extended_applications = [
    {'ticker': 'SBER', 'amount': 50000000, 'rate': 0.09},
    {'ticker': 'GAZP', 'amount': 100000000, 'rate': 0.10},
    {'ticker': 'VTBR', 'amount': 80000000, 'rate': 0.12},
    {'ticker': 'GMKN', 'amount': 150000000, 'rate': 0.11},
    {'ticker': 'LNTA', 'amount': 30000000, 'rate': 0.15},
    {'ticker': 'LKOH', 'amount': 120000000, 'rate': 0.105},
    {'ticker': 'ROSN', 'amount': 90000000, 'rate': 0.108},
    {'ticker': 'NLMK', 'amount': 70000000, 'rate': 0.13},
    {'ticker': 'MTSS', 'amount': 60000000, 'rate': 0.14},
    {'ticker': 'MGNT', 'amount': 40000000, 'rate': 0.16},
]

# Запускаем оптимизацию
optimal_portfolio = credit_portfolio.optimize_credit_portfolio(
    loan_applications=extended_applications,
    portfolio_budget=1000000000,  # 1 млрд рублей
    target_return=0.12  # Целевая доходность 12%
)

print("🎯 РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ:")
print("=" * 60)
print(f"💰 Общий бюджет: {optimal_portfolio['portfolio_budget']:,} руб. ({optimal_portfolio['portfolio_budget']/1e6:.0f} млн)")
print(f"💸 Размещено средств: {optimal_portfolio['total_allocated']:,} руб. ({optimal_portfolio['total_allocated']/1e6:.1f} млн)")
print(f"📊 Коэффициент использования: {optimal_portfolio['utilization_rate']:.1%}")
print(f"📈 Портфельная доходность: {optimal_portfolio['portfolio_return']:.2%}")
print(f"⚠️  Портфельная PD: {optimal_portfolio['portfolio_pd']:.3f} ({optimal_portfolio['portfolio_pd']*100:.2f}%)")
print(f"💹 Спрэд (Доходность - PD): {optimal_portfolio['portfolio_spread']:.2%}")
print(f"🔢 Количество кредитов: {optimal_portfolio['number_of_loans']}")
print(f"❌ Отклоненных заявок: {optimal_portfolio['rejected_applications']}")

# Детальный состав портфеля
if optimal_portfolio['selected_loans']:
    print(f"\n📋 СОСТАВ ОПТИМАЛЬНОГО ПОРТФЕЛЯ:")
    print("-" * 60)
    print(f"{'Компания':<8} {'Сумма (млн)':<12} {'Доля':<8} {'Ставка':<8} {'PD':<8} {'Сектор':<12}")
    print("-" * 60)
    
    for loan in optimal_portfolio['selected_loans']:
        amount_mln = loan['amount'] / 1e6
        share = loan['amount'] / optimal_portfolio['total_allocated']
        print(f"{loan['borrower']:<8} {amount_mln:<12.1f} {share:<8.1%} {loan['recommended_rate']:<8.1%} {loan['pd']:<8.3f} {loan['sector']:<12}")

# Анализ по секторам
print(f"\n🏭 РАСПРЕДЕЛЕНИЕ ПО СЕКТОРАМ:")
print("-" * 40)
for sector, allocation in optimal_portfolio['sector_allocation'].items():
    print(f"{sector:<15}: {allocation:.1%}")

# Визуализация оптимального портфеля
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Распределение по компаниям
if optimal_portfolio['selected_loans']:
    companies = [loan['borrower'] for loan in optimal_portfolio['selected_loans']]
    amounts = [loan['amount']/1e6 for loan in optimal_portfolio['selected_loans']]
    
    axes[0,0].pie(amounts, labels=companies, autopct='%1.1f%%')
    axes[0,0].set_title('Распределение кредитов по компаниям (млн руб.)')

# 2. Распределение по секторам
sector_amounts = {}
for loan in optimal_portfolio['selected_loans']:
    sector = loan['sector']
    sector_amounts[sector] = sector_amounts.get(sector, 0) + loan['amount']/1e6

axes[0,1].pie(sector_amounts.values(), labels=sector_amounts.keys(), autopct='%1.1f%%')
axes[0,1].set_title('Распределение кредитов по секторам')

# 3. PD vs Доходность
pds = [loan['pd'] for loan in optimal_portfolio['selected_loans']]
rates = [loan['recommended_rate'] for loan in optimal_portfolio['selected_loans']]
amounts_norm = [loan['amount']/max(amounts)/1e6*100 for loan in optimal_portfolio['selected_loans']]

scatter = axes[1,0].scatter(pds, rates, s=amounts_norm, alpha=0.7, c=range(len(pds)), cmap='viridis')
axes[1,0].set_xlabel('Probability of Default (PD)')
axes[1,0].set_ylabel('Рекомендуемая ставка')
axes[1,0].set_title('PD vs Доходность (размер = сумма кредита)')

# 4. Риск-доходность компонентов
risk_return = [(loan['pd'], loan['recommended_rate'] - loan['pd'], loan['borrower']) for loan in optimal_portfolio['selected_loans']]
risk_return.sort(key=lambda x: x[1], reverse=True)

companies_sorted = [item[2] for item in risk_return]
spreads = [item[1] for item in risk_return]

axes[1,1].bar(range(len(spreads)), spreads)
axes[1,1].set_xticks(range(len(companies_sorted)))
axes[1,1].set_xticklabels(companies_sorted, rotation=45)
axes[1,1].set_ylabel('Риск-скорректированная доходность')
axes[1,1].set_title('Спрэд по компаниям (Ставка - PD)')

plt.tight_layout()
plt.show()

print(f"\n✅ Оптимизация завершена. Создан портфель с доходностью {optimal_portfolio['portfolio_return']:.2%} и PD {optimal_portfolio['portfolio_pd']*100:.2f}%")

## 7. Мониторинг и управление рисками портфеля

In [ ]:
# Комплексный мониторинг состояния кредитного портфеля
print("🔍 МОНИТОРИНГ СОСТОЯНИЯ КРЕДИТНОГО ПОРТФЕЛЯ\n")

# Получаем отчет о состоянии портфеля
health_report = credit_portfolio.monitor_credit_portfolio_health()

print("📊 ОСНОВНЫЕ ПОКАЗАТЕЛИ ПОРТФЕЛЯ:")
print("=" * 50)
print(f"🏆 Оценка здоровья портфеля: {health_report['portfolio_health_score']:.1f}/100")

pd_stats = health_report['pd_statistics']
print(f"📈 Средняя PD: {pd_stats['mean_pd']:.3f} ({pd_stats['mean_pd']*100:.2f}%)")
print(f"📊 Медианная PD: {pd_stats['median_pd']:.3f} ({pd_stats['median_pd']*100:.2f}%)")
print(f"📉 Минимальная PD: {pd_stats['min_pd']:.3f} ({pd_stats['min_pd']*100:.2f}%)")
print(f"📈 Максимальная PD: {pd_stats['max_pd']:.3f} ({pd_stats['max_pd']*100:.2f}%)")
print(f"📏 Стандартное отклонение PD: {pd_stats['std_pd']:.3f}")
print(f"🏢 Количество компаний: {pd_stats['companies_count']}")

# Предупреждения о рисках
if health_report['risk_warnings']:
    print(f"\n⚠️  ПРЕДУПРЕЖДЕНИЯ О РИСКАХ ({len(health_report['risk_warnings'])} активных):")
    print("-" * 50)
    for warning in health_report['risk_warnings']:
        print(f"🚨 {warning['message']}")
        if 'companies' in warning and isinstance(warning['companies'], list):
            print(f"   Затронутые компании: {', '.join(warning['companies'])}")
        if 'max_pd' in warning:
            print(f"   Максимальная PD: {warning['max_pd']:.3f} ({warning['max_pd']*100:.2f}%)")
else:
    print(f"\n✅ СТАТУС РИСКОВ: Критических предупреждений нет")

# Секторный анализ  
print(f"\n🏭 АНАЛИЗ ПО СЕКТОРАМ:")
print("-" * 50)
sector_analysis = health_report['sector_analysis']

sector_df = pd.DataFrame({
    'Средняя_PD': sector_analysis['mean'],
    'Макс_PD': sector_analysis['max'], 
    'Количество': sector_analysis['count']
})

print(sector_df.round(3))

# Рекомендации
print(f"\n💡 РЕКОМЕНДАЦИИ ПО УПРАВЛЕНИЮ:")
print("-" * 50)
if health_report['recommendations']:
    for i, rec in enumerate(health_report['recommendations'], 1):
        print(f"{i}. {rec}")
else:
    print("✅ Портфель находится в хорошем состоянии. Дополнительных мер не требуется.")

# Визуализация мониторинга
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Динамика PD портфеля
monthly_avg_pd = credit_portfolio.portfolio.groupby('date')['PD'].mean()
axes[0,0].plot(monthly_avg_pd.index, monthly_avg_pd.values, marker='o', linewidth=2, color='red')
axes[0,0].axhline(y=0.05, color='orange', linestyle='--', label='Лимит PD (5%)')
axes[0,0].set_title('Динамика средней PD портфеля')
axes[0,0].set_ylabel('Средняя PD')
axes[0,0].legend()
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Распределение компаний по уровню риска
current_pds = credit_portfolio.portfolio.groupby('ticker')['PD'].last()
risk_categories = pd.cut(current_pds, bins=[0, 0.01, 0.02, 0.03, 0.05, 1], 
                        labels=['Очень низкий\n(≤1%)', 'Низкий\n(1-2%)', 'Средний\n(2-3%)', 
                               'Повышенный\n(3-5%)', 'Высокий\n(>5%)'])
risk_counts = risk_categories.value_counts()

axes[0,1].bar(range(len(risk_counts)), risk_counts.values, color=['green', 'lightgreen', 'yellow', 'orange', 'red'])
axes[0,1].set_xticks(range(len(risk_counts)))
axes[0,1].set_xticklabels(risk_counts.index, rotation=45)
axes[0,1].set_title('Распределение компаний по уровню риска')
axes[0,1].set_ylabel('Количество компаний')

# 3. Секторные риски
sector_mean_pd = sector_df['Средняя_PD']
colors = ['red' if pd > 0.05 else 'orange' if pd > 0.03 else 'green' for pd in sector_mean_pd]
axes[0,2].bar(range(len(sector_mean_pd)), sector_mean_pd.values, color=colors)
axes[0,2].axhline(y=0.05, color='red', linestyle='--', alpha=0.7, label='Критический уровень')
axes[0,2].set_xticks(range(len(sector_mean_pd)))
axes[0,2].set_xticklabels(sector_mean_pd.index, rotation=45)
axes[0,2].set_title('Средняя PD по секторам')
axes[0,2].set_ylabel('Средняя PD')
axes[0,2].legend()

# 4. Корреляция между компаниями (риск концентрации)
company_returns = credit_portfolio.portfolio.pivot(index='date', columns='ticker', values='PD').corr()
im = axes[1,0].imshow(company_returns.values, cmap='coolwarm', vmin=-1, vmax=1)
axes[1,0].set_xticks(range(len(company_returns.columns)))
axes[1,0].set_yticks(range(len(company_returns.index)))
axes[1,0].set_xticklabels(company_returns.columns, rotation=45)
axes[1,0].set_yticklabels(company_returns.index)
axes[1,0].set_title('Корреляция PD между компаниями')
plt.colorbar(im, ax=axes[1,0])

# 5. Макроэкономическое влияние
macro_influence = credit_portfolio.portfolio[['PD', 'inflation', 'unemployment', 'usd_rub']].corr()['PD'].drop('PD')
axes[1,1].bar(range(len(macro_influence)), macro_influence.values, 
              color=['red' if x > 0 else 'blue' for x in macro_influence.values])
axes[1,1].set_xticks(range(len(macro_influence)))
axes[1,1].set_xticklabels(['Инфляция', 'Безработица', 'USD/RUB'], rotation=45)
axes[1,1].set_title('Влияние макрофакторов на PD')
axes[1,1].set_ylabel('Корреляция')
axes[1,1].axhline(y=0, color='black', linestyle='-', alpha=0.3)

# 6. Оценка здоровья портфеля
health_components = {
    'PD Level': max(0, 100 - pd_stats['mean_pd'] * 2000),
    'Diversification': max(0, 100 - pd_stats['std_pd'] * 1000),
    'Stability': max(0, min(100, 80 + np.random.normal(0, 10))),  # Примерная оценка
}

axes[1,2].bar(range(len(health_components)), health_components.values(), 
              color=['green' if v >= 70 else 'orange' if v >= 50 else 'red' for v in health_components.values()])
axes[1,2].set_xticks(range(len(health_components)))
axes[1,2].set_xticklabels(health_components.keys(), rotation=45)
axes[1,2].set_title(f'Компоненты здоровья портфеля\n(Общая оценка: {health_report["portfolio_health_score"]:.1f})')
axes[1,2].set_ylabel('Оценка (0-100)')
axes[1,2].set_ylim(0, 100)
axes[1,2].axhline(y=70, color='green', linestyle='--', alpha=0.5, label='Хорошо')
axes[1,2].axhline(y=50, color='orange', linestyle='--', alpha=0.5, label='Приемлемо')

plt.tight_layout()
plt.show()

print(f"\n✅ Мониторинг завершен. Статус портфеля: {'ЗДОРОВЫЙ' if health_report['portfolio_health_score'] >= 70 else 'ТРЕБУЕТ ВНИМАНИЯ' if health_report['portfolio_health_score'] >= 50 else 'КРИТИЧЕСКИЙ'}")

## 8. Генерация итогового отчета

In [ ]:
# Генерация комплексного отчета по управлению кредитным портфелем
print("📊 Генерация итогового отчета по управлению кредитным портфелем...\n")

# Генерируем отчет с включением решений по кредитам
credit_portfolio.generate_credit_report(
    loan_decisions=loan_decisions,  # Используем решения из предыдущих шагов
    save_path="./logs/credit_portfolio_report.txt"
)

## Заключение

### Ключевые достижения проекта:

1. **📊 Модель Мертона с макрофакторами**: Успешно интегрированы инфляция, безработица и курс USD/RUB для более точного расчета PD

2. **🎯 Автоматизированная система решений**: Создана система принятия кредитных решений на основе:
   - Лимитов PD (≤5%)
   - Секторных концентраций (≤25%)
   - Лимитов на одного заемщика (≤8%)

3. **⚖️ Оптимизация портфеля**: Реализована оптимизация состава кредитного портфеля с учетом риск-доходности

4. **🔍 Система мониторинга**: Разработан инструмент раннего предупреждения рисков с автоматической генерацией рекомендаций

### Практическое применение:

- **Банки**: Автоматизация кредитного андеррайтинга
- **Кредитные организации**: Управление концентрационными рисками  
- **Регуляторы**: Мониторинг системных рисков
- **Инвесторы**: Оценка кредитных портфелей

### Направления развития:

1. Интеграция дополнительных макроэкономических факторов
2. Машинное обучение для калибровки модели
3. Стресс-тестирование при экстремальных сценариях
4. API для интеграции с банковскими системами